In [2]:
%load_ext autoreload
%autoreload 2

In [26]:
from vggish import mel_features
from vggish import vggish_input
from vggish import vggish_slim

ModuleNotFoundError: No module named 'jinahub'

### Prepare vvgish input manually

In [4]:
import librosa

In [5]:
x_audio, sample_rate = librosa.load('data/sample.mp3')

/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [6]:
log_mel_examples = vggish_input.waveform_to_examples(x_audio, sample_rate)

In [7]:
log_mel_examples.shape

(28, 96, 64)

### We can directly go from path of mp3 file to input vggish

In [14]:
log_mel_examples = vggish_input.mp3_to_examples('data/sample.mp3')

/usr/local/lib/python3.9/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [18]:
log_mel_examples.shape

(28, 96, 64)

### Passing data thorugh the vvgish

In [20]:
import tensorflow as tf

In [21]:
sess = tf.compat.v1.Session()

In [ ]:
vggish/vggish_slim.def

In [27]:
net = define_vggish_slim()

NameError: name 'define_vggish_slim' is not defined

In [31]:

from jina import Executor

class VggishEncoder(Executor):
    def __init__(self, model_path: str, pca_path: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model_path = model_path
        self.pca_path = pca_path

        import tensorflow as tf
        tf.compat.v1.disable_eager_execution()
        self.sess = tf.compat.v1.Session()
        define_vggish_slim()
        load_vggish_slim_checkpoint(self.sess, self.model_path)
        self.feature_tensor = self.sess.graph.get_tensor_by_name(
            INPUT_TENSOR_NAME)
        self.embedding_tensor = self.sess.graph.get_tensor_by_name(
            OUTPUT_TENSOR_NAME)
        self.post_processor = Postprocessor(self.pca_path)

    def encode(self, content, *args, **kwargs):
        [embedding_batch] = self.sess.run([self.embedding_tensor],
                                          feed_dict={self.feature_tensor: content})
        result = self.post_processor.postprocess(embedding_batch)
        return (np.float32(result) - 128.) / 128.

In [ ]:
VggishEncoder(model_path=)